# NON-LINEAR ML TECHNIQUES

In [ ]:
# libraries
import pandas as pd 
import math
import numpy as np 
import matplotlib.pyplot as plt
import datetime
from sklearn import metrics
import lightgbm as lgb
from sktime.forecasting.base import ForecastingHorizon
from sktime.transformations.series.detrend import Deseasonalizer, Detrender
from sktime.forecasting.trend import PolynomialTrendForecaster
from sktime.utils.plotting import plot_series
from sktime.forecasting.compose import TransformedTargetForecaster,ReducedForecaster
from sktime.forecasting.compose import make_reduction
from sklearn.ensemble import AdaBoostRegressor
import xgboost as xg
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# In order to use this notebook for univarate time series analysis :-
# 1) The primary requirement is not to have missing values or categorial(string) data for time_dependent variable 
#    and time_column.
# 2) This cell requires information on file_name (only csv), time_dependent_variable, time_column, date_time format (frmt)
#    and resample grain(X). After filling the required information correctly, you can run all the cells (Cell ---> Run All)
# 3) Example :-
#   file_name               = "JetRail Avg Hourly Traffic Data - 2012-2013.csv"
#   time_dependent_variable = "Count"    (column name in your dataset)
#   time_column             = "Datetime" (column name in your dataset)
#   frmt                    = "%Y-%m-%d"
#   X                       = "D" 

file_name = "Monthly Production of Chocolate - Australia.csv"
time_dependent_variable = "Volume"
time_column = "Month"
frmt =  '%Y-%m'
X = "M"

### Reading the csv file

In [ ]:
def data(time_column, file_name, frmt='%Y-%m-%d %H:%M:%S', X= "D"):
    df = pd.read_csv(file_name, parse_dates= True)
    df = df[[time_column,time_dependent_variable]]
    df[time_column] = pd.to_datetime(df[time_column],format=frmt) 
    df.index = df[time_column]
    df = df.resample(X).mean()
    df.reset_index(inplace= True)
    return df
df = data(time_column, file_name, frmt, X)

In [ ]:
df.head()

## Defining the metrics

In [ ]:
def timeseries_evaluation_metrics_func(y_true, y_pred):
    def mean_absolute_percentage_error(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    print('Evaluation metric results:-')
    print(f'MSE is : {metrics.mean_squared_error(y_true, y_pred)}')
    print(f'MAE is : {metrics.mean_absolute_error(y_true, y_pred)}')
    print(f'RMSE is : {np.sqrt(metrics.mean_squared_error(y_true, y_pred))}')
    print(f'MAPE is : {mean_absolute_percentage_error(y_true,y_pred)}')
    print(f'R2 is : {metrics.r2_score(y_true, y_pred)}',end='\n\n')

## Plot Function

In [ ]:
def plot():
    plt.figure(figsize=(12,8))
    plt.plot(train.index, train[time_dependent_variable], label='Train')
    plt.plot(test.index,test[time_dependent_variable], label='Test')
    plt.plot(test.index,y_pred, label= 'forecast')
    plt.legend(loc='best')
    plt.title('forecast')
    plt.show()
    timeseries_evaluation_metrics_func(test[time_dependent_variable],y_pred)

In [ ]:
def plot_ml():
    plt.figure(figsize=(12,8))
    plt.plot(train.index, train[time_dependent_variable], label='Train')
    plt.plot(test.index,test[time_dependent_variable], label='Test')
    plt.plot(test.index,pred, label= 'forecast')
    plt.legend(loc='best')
    plt.title('forecast')
    plt.show()
    timeseries_evaluation_metrics_func(test_Y,pred)

## Feature Engineering
To use supervised machine learning, we need input and output variables. We can create features from a univariate time series in the following ways :-
- Date Time Features : These are components of the time step itself for each observation.
- Lag Features : These are values at prior time steps.
- Window Features : These are a summary of values over a fixed window of prior time steps.

let's try and explore each one of them...

#### 1) Date Time Features

In [ ]:
def date_features(df):
    df['year'] = df[time_column].dt.year
    df['quarter'] = df[time_column].dt.quarter
    df['month'] = df[time_column].dt.month
    df['week_day'] = df[time_column].dt.weekday
    return df
df_date_features = date_features(df)[["year","quarter","month","week_day",time_dependent_variable]]

#### 2) Lag Features 

In [ ]:
df_lag_features = pd.DataFrame()

def lag_features(n):
    df_lag_features["lag_{}".format(n)] = df[time_dependent_variable].shift(n)

Let's take lag = 1 and lag = 2 as features. It totally depends on your problem......

In [ ]:
# if you want a different lag k just write lag_features(k) and the function would automatically do it for you..
lag_features(1)
lag_features(2)

#### 3) Window Features 

In [ ]:
df_window_features = pd.DataFrame()

def rolling_mean(n):
    df_window_features["Rolling_mean_{}".format(n)] = df[time_dependent_variable].shift(1).rolling(window = n).mean()

Let's take rolling mean of 2 and 5. It totally depends on your problem.....

In [ ]:
# if you want a different rolling mean k just write rolling_mean(k) and the function would automatically do it for you..
rolling_mean(2)
rolling_mean(5)

#### Combining the table with features 

In [ ]:
df = pd.merge(pd.merge(df_date_features,df_lag_features,left_index=True,right_index=True),df_window_features,left_index=True, right_index=True)
df = df.dropna()
df.head()

### Splitting the data into train and test using (you can use any one of them) :-

In [ ]:
# This splits the data into train and test using default split_size = 0.7
def train_test_split_perc(df, split= 0.7):
    total_size=len(df)
    train_size=math.floor(split*total_size) #(70% Dataset)
    train = df.head(train_size)
    test  = df.tail(len(df) - train_size)
    return train,test
    
train,test = train_test_split_perc(df, split= 0.9)

In [ ]:
train_Y = train[time_dependent_variable]
train_X = train[[i for i in df.columns if i != time_dependent_variable]]
test_Y = test[time_dependent_variable]
test_X = test[[i for i in df.columns if i != time_dependent_variable]]

In [ ]:
len(train_Y) == len(train_X)

### Model Function

In [ ]:
def model(algorithm):
    forecaster = TransformedTargetForecaster([("deseasonalise", Deseasonalizer(model="multiplicative", sp=12)),
                                          ("detrend", Detrender(forecaster=PolynomialTrendForecaster(degree=2))),
                                          ("forecast",make_reduction(algorithm.fit(train_X, train_Y), window_length=5, strategy="recursive"))])
    fh = ForecastingHorizon(test.index, is_relative=False)
    forecaster.fit(train[time_dependent_variable])
    y_pred = forecaster.predict(fh)
    plt.figure(figsize=(12,8))
    plt.plot(train.index, train[time_dependent_variable], label='Train')
    plt.plot(test.index,test[time_dependent_variable], label='Test')
    plt.plot(test.index,y_pred, label= 'forecast')
    plt.legend(loc='best')
    plt.title('forecast')
    plt.show()
    timeseries_evaluation_metrics_func(test[time_dependent_variable],y_pred)

# MODELS

### 1) LightGBM (univariate)
##### Pipelining, detrending and deseasonalization<a class="anchor" id="section_3_2"></a>

A common composition motive is pipelining: for example, first deseasonalizing or detrending the data, then forecasting the
detrended/deseasonalized series. When forecasting, one needs to add the trend and seasonal component back to the data. 

Create Pipeline :-
- Separate the Seasonal Component.
- Fit a forecaster for the trend.
- Fit a Autoregressor to the resdiual(autoregressing on four historic values).

In [ ]:
def get_transformed_target_forecaster(alpha,params):
    
    #Initialize Light GBM Regressor 
    
    regressor = lgb.LGBMRegressor(alpha = alpha,**params)
    
    forecaster = TransformedTargetForecaster([("deseasonalise", Deseasonalizer(model="multiplicative", sp=12)),
                                              ("detrend", Detrender(forecaster=PolynomialTrendForecaster(degree=2))),
                                            ("forecast",make_reduction(regressor, window_length=5, strategy="recursive"))])
    return forecaster

In [ ]:
# ------------------Fitting an Auto Regressive Light-GBM------------
# Objectives can be :- regression, regression_l1, huber, fair, poisson, quantile, mape, gamma, tweedie, binary, multiclass,
# multiclassova, cross_entropy, cross_entropy_lambda, lambdarank, rank_xendcg  

# Setting objective as 'mape' here.

params = {'objective':'quantile'}
                               #A 10 percent and 90 percent prediction interval(0.1,0.9 respectively).
alpha_params = [.1, .5, .9]    # Hyper-parameter "alpha" in Light GBM
                               #Capture forecasts for 10th/median/90th quantile, respectively.
for alpha in alpha_params:
    
    forecaster = get_transformed_target_forecaster(alpha,params)
    
    #Initialize ForecastingHorizon class to specify the horizon of forecast
    fh = ForecastingHorizon(test.index, is_relative=False)
    
    #Fit on Training data.
    forecaster.fit(train[time_dependent_variable])
    
    #Forecast the values.
    y_pred = forecaster.predict(fh)
    plot()


Now let's use lightGBM as a normal machine learning technique with input and output variables....

In [ ]:
rg = lgb.LGBMRegressor().fit(train_X, train_Y)
pred = rg.predict(test_X)
plot_ml()

### 2) Adaboost

In [ ]:
rg = AdaBoostRegressor().fit(train_X, train_Y)
pred = rg.predict(test_X)
plot_ml()

In [ ]:
model(AdaBoostRegressor())

### 3) XGBoost

In [ ]:
rg = xg.XGBRegressor().fit(train_X, train_Y)
pred = rg.predict(test_X)
plot_ml()

In [ ]:
model(xg.XGBRegressor())

### 4) Bagged Decision Trees

In [ ]:
rg = BaggingRegressor().fit(train_X, train_Y)
pred = rg.predict(test_X)
plot_ml()

In [ ]:
model(BaggingRegressor())

### 5) GradientBoostingRegressor

In [ ]:
rg = GradientBoostingRegressor().fit(train_X, train_Y)
pred = rg.predict(test_X)
plot_ml()

In [ ]:
model(GradientBoostingRegressor())

### 6) Extra TreesRegressor

In [ ]:
rg = ExtraTreesRegressor().fit(train_X, train_Y)
pred = rg.predict(test_X)
plot_ml()

In [ ]:
model(ExtraTreesRegressor())

### 7) Decision Tree Regressor

In [ ]:
rg = DecisionTreeRegressor().fit(train_X, train_Y)
pred = rg.predict(test_X)
plot_ml()

In [ ]:
model(DecisionTreeRegressor())

### 8) Support Vector Regression

In [ ]:
rg = make_pipeline(StandardScaler(), SVR(C=1.0, epsilon=0.2)).fit(train_X, train_Y)
pred = rg.predict(test_X)
plot_ml()

### 9) KNN Regression

In [ ]:
rg = KNeighborsRegressor(n_neighbors=5).fit(train_X, train_Y)
pred = rg.predict(test_X)
plot_ml()

In [ ]:
model(KNeighborsRegressor(n_neighbors=5))

### 10) Random Forest Regression 

In [ ]:
rg = RandomForestRegressor().fit(train_X, train_Y)
pred = rg.predict(test_X)
plot_ml()

In [ ]:
model(RandomForestRegressor())